<h1>Crypto Market Simulator</h1>

In [69]:
import asyncio
import ujson
import time
import sys 
import pandas as pd

from gate_ws import Configuration, Connection, WebSocketResponse
from gate_ws.spot import SpotPublicTradeChannel, SpotOrderBookChannel, SpotOrderBookUpdateChannel

In [70]:
NUM_OF_SNAPSHOTS = 5
curr_snapshot = 0
orders = []

<h2>Function to get orderbook data in json format</h2>

In [71]:
# define your callback function on message received
def print_message(conn: Connection, response: WebSocketResponse):
    global curr_snapshot
    global NUM_OF_SNAPSHOTS
    global orders
    if response.error:
        print('error returned: ', response.error)
        conn.close()
        return
    
    data = response.result
    if list(data.keys())[0] != "status":
        orders.append(data)
        curr_snapshot += 1
    
    if curr_snapshot >= NUM_OF_SNAPSHOTS:
        conn.close()
        return
        
async def main():
    channel_name = "BTC_USD"
    depth = "20"
    update_rate = "1000ms"
    
    filename = "orderbook.json"
    #initialize default connection, which connects to spot WebSocket V4
    # it is recommended to use one conn to initialize multiple channels
    conn = Connection(Configuration())

    # subscribe to any channel you are interested into, with the callback function
    channel = SpotOrderBookChannel(conn, print_message)
    
    # start_time = time.perf_counter()
    
    channel.subscribe([channel_name, depth, update_rate])
    
    # end_time = time.perf_counter()
    
    # print(f"Time: {end_time - start_time}")

    # start the client
    await conn.run()
    
    with open(filename, "w") as file:
        data = ujson.dumps(orders, indent=4)
        print(data)
        file.write(data)

if __name__ == '__main__':
    start_time = time.perf_counter()
    await main()
    end_time = time.perf_counter()
    print(f"Time: {end_time - start_time}")

[
    {
        "t": 1671278671573,
        "lastUpdateId": 860766267,
        "s": "BTC_USD",
        "bids": [
            [
                "16699.43",
                "0.0247"
            ],
            [
                "16699.42",
                "0.0029"
            ],
            [
                "16697.3",
                "0.3002"
            ],
            [
                "16693.45",
                "0.0078"
            ],
            [
                "16689.34",
                "0.0617"
            ],
            [
                "16689",
                "0.5413"
            ],
            [
                "16676.01",
                "0.5403"
            ],
            [
                "16676",
                "0.0519"
            ],
            [
                "16674.77",
                "0.011"
            ],
            [
                "16666.54",
                "0.0094"
            ],
            [
                "16662.67",
                "0.0291"
        

In [72]:
df = pd.read_json("orderbook.json")

df

,t,lastUpdateId,s,bids,asks
0,1671278671573,860766267,BTC_USD,"[[16699.43, 0.0247], [16699.42, 0.0029], [1669...","[[16715.89, 0.025], [16715.9, 0.3002], [16716...."
1,1671278672724,860766345,BTC_USD,"[[16702.6, 0.3002], [16699.43, 0.0247], [16699...","[[16716.29, 0.025], [16716.3, 0.005], [16716.3..."
2,1671278673919,860766451,BTC_USD,"[[16702.61, 0.025], [16702.6, 0.3002], [16699....","[[16716.29, 0.025], [16716.3, 0.005], [16716.3..."
3,1671278675023,860766515,BTC_USD,"[[16702.62, 0.0017], [16702.61, 0.025], [16702...","[[16716.29, 0.025], [16716.3, 0.005], [16716.3..."
4,1671278675926,860766590,BTC_USD,"[[16702.61, 0.025], [16702.6, 0.3002], [16699....","[[16716.28, 0.0061], [16716.29, 0.025], [16716..."


In [73]:
df2 = pd.DataFrame(df)
df2[[f'bid_{i}' for i in range(20)]] = pd.DataFrame(df2.bids.tolist(), index= df2.index)
df2[[f'ask_{i}' for i in range(20)]] = pd.DataFrame(df2.asks.tolist(), index= df2.index)

for i in range(20):
    df2[[f'bid_{i}_px', f"bid_{i}_qty"]] = pd.DataFrame(df2[f"bid_{i}"].tolist(), index= df2.index)
    df2[[f'ask_{i}_px', f"ask_{i}_qty"]] = pd.DataFrame(df2[f"ask_{i}"].tolist(), index= df2.index)
    

df2

/Users/kritiksatija/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]


,t,lastUpdateId,s,bids,asks,bid_0,bid_1,bid_2,bid_3,bid_4,...,ask_17_px,ask_17_qty,bid_18_px,bid_18_qty,ask_18_px,ask_18_qty,bid_19_px,bid_19_qty,ask_19_px,ask_19_qty
0,1671278671573,860766267,BTC_USD,"[[16699.43, 0.0247], [16699.42, 0.0029], [1669...","[[16715.89, 0.025], [16715.9, 0.3002], [16716....","[16699.43, 0.0247]","[16699.42, 0.0029]","[16697.3, 0.3002]","[16693.45, 0.0078]","[16689.34, 0.0617]",...,16716.84,0.005,16596.22,0.0584,16716.86,0.005,16594.63,1.0878,16716.88,0.005
1,1671278672724,860766345,BTC_USD,"[[16702.6, 0.3002], [16699.43, 0.0247], [16699...","[[16716.29, 0.025], [16716.3, 0.005], [16716.3...","[16702.6, 0.3002]","[16699.43, 0.0247]","[16699.42, 0.0029]","[16692.79, 0.0035]","[16689.34, 0.0617]",...,16716.86,0.005,16596.22,0.0584,16716.88,0.005,16594.63,1.0878,16716.9,0.005
2,1671278673919,860766451,BTC_USD,"[[16702.61, 0.025], [16702.6, 0.3002], [16699....","[[16716.29, 0.025], [16716.3, 0.005], [16716.3...","[16702.61, 0.025]","[16702.6, 0.3002]","[16699.42, 0.0029]","[16695.29, 0.0035]","[16689.34, 0.0617]",...,16716.86,0.005,16596.22,0.0584,16716.88,0.005,16594.63,1.0878,16716.9,0.005
3,1671278675023,860766515,BTC_USD,"[[16702.62, 0.0017], [16702.61, 0.025], [16702...","[[16716.29, 0.025], [16716.3, 0.005], [16716.3...","[16702.62, 0.0017]","[16702.61, 0.025]","[16702.6, 0.3002]","[16699.42, 0.0029]","[16695.81, 0.0035]",...,16716.86,0.005,16596.22,0.0584,16716.88,0.005,16594.63,1.0878,16716.9,0.005
4,1671278675926,860766590,BTC_USD,"[[16702.61, 0.025], [16702.6, 0.3002], [16699....","[[16716.28, 0.0061], [16716.29, 0.025], [16716...","[16702.61, 0.025]","[16702.6, 0.3002]","[16699.42, 0.0029]","[16695.81, 0.0035]","[16689.34, 0.0617]",...,16716.84,0.005,16596.22,0.0584,16716.86,0.005,16594.63,1.0878,16716.88,0.005


<h2>Final dataframe format</h2>

In [74]:
#Remove unnecessary columns
df2.drop(columns = [f'bid_{i}' for i in range(20)]+[f'ask_{i}' for i in range(20)]+["bids","asks", "lastUpdateId"], axis=1, inplace = True)

df2

,t,s,bid_0_px,bid_0_qty,ask_0_px,ask_0_qty,bid_1_px,bid_1_qty,ask_1_px,ask_1_qty,...,ask_17_px,ask_17_qty,bid_18_px,bid_18_qty,ask_18_px,ask_18_qty,bid_19_px,bid_19_qty,ask_19_px,ask_19_qty
0,1671278671573,BTC_USD,16699.43,0.0247,16715.89,0.025,16699.42,0.0029,16715.9,0.3002,...,16716.84,0.005,16596.22,0.0584,16716.86,0.005,16594.63,1.0878,16716.88,0.005
1,1671278672724,BTC_USD,16702.6,0.3002,16716.29,0.025,16699.43,0.0247,16716.3,0.005,...,16716.86,0.005,16596.22,0.0584,16716.88,0.005,16594.63,1.0878,16716.9,0.005
2,1671278673919,BTC_USD,16702.61,0.025,16716.29,0.025,16702.6,0.3002,16716.3,0.005,...,16716.86,0.005,16596.22,0.0584,16716.88,0.005,16594.63,1.0878,16716.9,0.005
3,1671278675023,BTC_USD,16702.62,0.0017,16716.29,0.025,16702.61,0.025,16716.3,0.005,...,16716.86,0.005,16596.22,0.0584,16716.88,0.005,16594.63,1.0878,16716.9,0.005
4,1671278675926,BTC_USD,16702.61,0.025,16716.28,0.0061,16702.6,0.3002,16716.29,0.025,...,16716.84,0.005,16596.22,0.0584,16716.86,0.005,16594.63,1.0878,16716.88,0.005
